# Data Chunking with Dask

In this notebook we demonstrate:

* Xarray + Dask
* NetCDF file Chunks versus Dask Chunks
* chunk shapes

The following material uses Coupled Model Intercomparison Project (CMIP6) collections. Please visit data collection [catalogue](https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f6600_2266_8675_3563) and [CMIP6 terms of use](https://pcmdi.llnl.gov/CMIP6/TermsOfUse/TermsOfUse6-1.html) for more information. 

---

- Authors: NCI Virtual Research Environment Team
- Keywords: CMIP6, Xarray, Dask, Diagnostics
- Create Date: 2019-June; Update Date: 2020-May

### Prerequisite

You need to run this notebook on Gadi/VDI (recommended), or on your local computer if CMIP6 data can be accessed from remote links via data services. The following modules are needed:

* Xarray
* Dask
* NetCDF

You also need to be a member of the following data project to access the data or through [NCI's data service](http://dapds00.nci.org.au/thredds/catalog.html) if the data is published:
* oi10

You can request to join the project through [NCI's user account management system](https://my.nci.org.au). 

### Load the required modules

In [10]:
import xarray as xr
import netCDF4 as nc
import time
%matplotlib inline

Starting the Dask Client is optional. It will provide a dashboard which is useful to gain insight on the computation.

The link to the dashboard will become visible when you create the client below. We recommend having it open on one side of your screen while using your notebook on the other side. This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [3]:
# Create cluster
from dask.distributed import Client,LocalCluster
client = Client(scheduler_file='scheduler.json')
print(client)

<Client: 'tcp://10.6.52.21:8731' processes=32 threads=32, memory=68.72 GB>


### Data

We will use percipitation data update of RCP8.5 based on SSP5 from GFDL-CM4 ensembles in this example, let's have a look at the data.

In [4]:
!ncdump -hst '/g/data/oi10/replicas/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-CM4/ssp585/r1i1p1f1/day/pr/gr1/v20180701/pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20150101-20341231.nc'

netcdf pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20150101-20341231 {
dimensions:
	lat = 180 ;
	bnds = 2 ;
	lon = 288 ;
	time = UNLIMITED ; // (7300 currently)
variables:
	double lat(lat) ;
		lat:long_name = "latitude" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
		lat:bounds = "lat_bnds" ;
		lat:standard_name = "latitude" ;
		lat:cell_methods = "time: point" ;
		lat:_Storage = "chunked" ;
		lat:_ChunkSizes = 180 ;
		lat:_DeflateLevel = 2 ;
		lat:_Shuffle = "true" ;
		lat:_Endianness = "little" ;
	double lat_bnds(lat, bnds) ;
		lat_bnds:long_name = "latitude bounds" ;
		lat_bnds:units = "degrees_north" ;
		lat_bnds:axis = "Y" ;
		lat_bnds:_Storage = "chunked" ;
		lat_bnds:_ChunkSizes = 180, 2 ;
		lat_bnds:_DeflateLevel = 2 ;
		lat_bnds:_Shuffle = "true" ;
		lat_bnds:_Endianness = "little" ;
	double lon(lon) ;
		lon:long_name = "longitude" ;
		lon:units = "degrees_east" ;
		lon:axis = "X" ;
		lon:bounds = "lon_bnds" ;
		lon:standard_name = "longitude" ;
		lon:cell_methods = "time: poin

### Xarray + Dask

Xarray can automatically wrap its data in dask arrays. This capability turns xarray into an extremely powerful tool for Big Data earth science

To see this in action, we will download a fairly large dataset to analyze. We use xarray's `open_mfdataset` allows multiple files to be opened simultaneously.

In [5]:
!ls /g/data/oi10/replicas/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-CM4/ssp585/r1i1p1f1/day/pr/gr1/v20180701
path = '/g/data/oi10/replicas/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-CM4/ssp585/r1i1p1f1/day/pr/gr1/v20180701/*'

pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20150101-20341231.nc
pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20350101-20541231.nc
pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20550101-20741231.nc
pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20750101-20941231.nc
pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20950101-21001231.nc


In [6]:
f_ssp585 = xr.open_mfdataset(path, combine='by_coords')
f_ssp585

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 180, lon: 288, time: 31390)
Coordinates:
  * lon        (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * time       (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
Dimensions without coordinates: bnds
Data variables:
    lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(7300, 180, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(7300, 288, 2), meta=np.ndarray>
    pr         (time, lat, lon) float32 dask.array<chunksize=(7300, 180, 288), meta=np.ndarray>
    time_bnds  (time, bnds) object dask.array<chunksize=(7300, 2), meta=np.ndarray>
Attributes:
    external_variables:     areacella
    history:                File was processed by fremetar (GFDL analog of CM...
    table_id:               day
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    comment:                <null ref>
    contact:                gfdl.climate.model.info@noaa.gov
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    creation_date:          2019-03-18T15:13:56Z
    data_specs_version:     01.00.27
    experiment:             update of RCP8.5 based on SSP5
    experiment_id:          ssp585
    forcing_index:          1
    frequency:              day
    further_info_url:       https://furtherinfo.es-doc.org/CMIP6.NOAA-GFDL.GF...
    grid:                   atmos data regridded from Cubed-sphere (c96) to 1...
    grid_label:             gr1
    initialization_index:   1
    institution:            National Oceanic and Atmospheric Administration, ...
    institution_id:         NOAA-GFDL
    license:                CMIP6 model data produced by NOAA-GFDL is license...
    mip_era:                CMIP6
    nominal_resolution:     100 km
    parent_activity_id:     CMIP
    parent_experiment_id:   historical
    parent_mip_era:         CMIP6
    parent_source_id:       GFDL-CM4
    parent_variant_label:   r1i1p1f1
    physics_index:          1
    product:                model-output
    realization_index:      1
    realm:                  atmos
    source:                 GFDL-CM4 (2018): \naerosol: interactive\natmos: G...
    source_id:              GFDL-CM4
    source_type:            AOGCM
    sub_experiment:         none
    sub_experiment_id:      none
    title:                  NOAA GFDL GFDL-CM4 model output prepared for CMIP...
    tracking_id:            hdl:21.14100/ad8c930c-124f-4b82-98b2-98cc4236ba25
    variable_id:            pr
    variant_info:           N/A
    references:             see further_info_url attribute
    variant_label:          r1i1p1f1
    branch_time_in_parent:  60225.0
    parent_time_units:      days since 1850-1-1

<div class="alert alert-warning">
<b>NOTE:</b> the values are not displayed, since that would trigger computation.. 
</div>

### Chunks

Notice that it says:`pr(time, lat, lon) float32 dask.array<chunksize=(7300, 180, 288), meta=np.ndarray>`. There is now the `chunksize` component. The data array also become an dask array.

The chunking of the array comes from the integration of Dask with xarray. Dask (see: https://docs.dask.org/en/latest/) is a library for parallel computing. Dask divides the data array into small pieces called "chunks", with each chunk designed to be small enough to fit into memory. 

The file itself may be already chunked. Filesystem chunking is available in netCDF-4 and HDF5 datasets. CMIP6 data should all be netCDF-4 and include some form of chunking on the file.

In [7]:
!ncdump -hst '/g/data/oi10/replicas/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-CM4/ssp585/r1i1p1f1/day/pr/gr1/v20180701/pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20150101-20341231.nc'

netcdf pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20150101-20341231 {
dimensions:
	lat = 180 ;
	bnds = 2 ;
	lon = 288 ;
	time = UNLIMITED ; // (7300 currently)
variables:
	double lat(lat) ;
		lat:long_name = "latitude" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
		lat:bounds = "lat_bnds" ;
		lat:standard_name = "latitude" ;
		lat:cell_methods = "time: point" ;
		lat:_Storage = "chunked" ;
		lat:_ChunkSizes = 180 ;
		lat:_DeflateLevel = 2 ;
		lat:_Shuffle = "true" ;
		lat:_Endianness = "little" ;
	double lat_bnds(lat, bnds) ;
		lat_bnds:long_name = "latitude bounds" ;
		lat_bnds:units = "degrees_north" ;
		lat_bnds:axis = "Y" ;
		lat_bnds:_Storage = "chunked" ;
		lat_bnds:_ChunkSizes = 180, 2 ;
		lat_bnds:_DeflateLevel = 2 ;
		lat_bnds:_Shuffle = "true" ;
		lat_bnds:_Endianness = "little" ;
	double lon(lon) ;
		lon:long_name = "longitude" ;
		lon:units = "degrees_east" ;
		lon:axis = "X" ;
		lon:bounds = "lon_bnds" ;
		lon:standard_name = "longitude" ;
		lon:cell_methods = "time: poin

#### In this case the file is chunked such that  `pr:_ChunkSizes = 1, 180, 288 ;`

Here we see that the data is chunked in time, where one chunk is one time-step and all points in lat-lon.

![](images/Chunks.png)
image source: https://www.unidata.ucar.edu/blogs/developer/en/entry/chunking_data_why_it_matters

Consider 2 types of data access
1. Accessing a 2D lat-lon slice in time (RHS figure)
2. Accessing a time series at a single lat-lon point (LHS figure)

With the chunking above, the first type of data access only needs to access a single chunk, while the second type needs to access ALL the chunks of the data array regardless. This dataset will be fastest for 2D lat-lon single time-step data access.

In general, even without chunking - when the data is accessed contiguously (by index order) - time is the slowest variable to access, then y, with x being the fastest. With the chunking method of this CMIP6 dataset, time still remains the slowest variable. For more uniform variable access speeds more evenly spaced chunks would be needed, spacing the chunks in time, lat, and lon.

### Exercise

Time how long it takes to load the precipitation data at `time='2015-01-01'` and then time how long it takes to load the data at `lat=0` and `lon=180` (remember to use `method='nearest'` for the latter case). How much difference is there in the different access methods?


In [12]:
%%time
f_ssp585.pr.sel(time='2015-01-01').load()

CPU times: user 28.8 ms, sys: 1.88 ms, total: 30.7 ms
Wall time: 435 ms


<xarray.DataArray 'pr' (time: 1, lat: 180, lon: 288)>
array([[[1.1679080e-07, 1.1679080e-07, 1.1679080e-07, ...,
         1.1679080e-07, 1.1679080e-07, 1.1679080e-07],
        [5.6357038e-08, 5.6899808e-08, 5.7449537e-08, ...,
         5.4770052e-08, 5.5292201e-08, 5.5821182e-08],
        [7.7667160e-08, 7.7346748e-08, 7.7024652e-08, ...,
         7.8618250e-08, 7.8302911e-08, 7.7985881e-08],
        ...,
        [7.0853553e-06, 7.2527428e-06, 7.4210134e-06, ...,
         6.5884897e-06, 6.7532287e-06, 6.9188504e-06],
        [2.2063125e-05, 2.2134822e-05, 2.2207438e-05, ...,
         2.1853495e-05, 2.1922468e-05, 2.1992342e-05],
        [3.0045994e-05, 3.0045994e-05, 3.0045994e-05, ...,
         3.0045994e-05, 3.0045994e-05, 3.0045994e-05]]], dtype=float32)
Coordinates:
  * lon      (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * time     (time) object 2015-01-01 12:00:00
Attributes:
    long_name:      Precipitation
    units:          kg m-2 s-1
    cell_methods:   area: time: mean
    cell_measures:  area: areacella
    standard_name:  precipitation_flux
    interp_method:  conserve_order1
    original_name:  pr

In [13]:
%%time
f_ssp585.pr.sel(lat=0,lon=180,method='nearest').load()

CPU times: user 43.5 ms, sys: 2.42 ms, total: 45.9 ms
Wall time: 15.4 s


<xarray.DataArray 'pr' (time: 31390)>
array([2.1776618e-06, 9.6000194e-06, 5.3295435e-06, ..., 7.7904500e-05,
       1.6186357e-05, 1.5706388e-05], dtype=float32)
Coordinates:
    lon      float64 180.6
    lat      float64 0.5
  * time     (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
Attributes:
    long_name:      Precipitation
    units:          kg m-2 s-1
    cell_methods:   area: time: mean
    cell_measures:  area: areacella
    standard_name:  precipitation_flux
    interp_method:  conserve_order1
    original_name:  pr

### Approximately the same amount of data took 100x longer to load.

The spatial dataset contains 51840 data-points and took order 100ms to load. The time-series dataset has 31390 data-points and took order 10,000 ms to load.

Chunking and the ways in which the data is read is important in considering both how you access the data and if you wish to parallelise your code.


### NetCDF file Chunks versus Dask Chunks

Keep in mind, dask chunking is different to chunking of the stored data. As we saw, the stored data is chunked with chunks of size (1,180,288). The Dask array was chunked with size (7300, 180, 288). You can change the chunking in the dask array. In the below example we are specifying that there be 730 chunks in time.

In [15]:
f_ssp585 = xr.open_mfdataset(path,chunks={'time':730}, combine='by_coords')

f_ssp585

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 180, lon: 288, time: 31390)
Coordinates:
  * lon        (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * time       (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
Dimensions without coordinates: bnds
Data variables:
    lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(7300, 180, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(7300, 288, 2), meta=np.ndarray>
    pr         (time, lat, lon) float32 dask.array<chunksize=(730, 180, 288), meta=np.ndarray>
    time_bnds  (time, bnds) object dask.array<chunksize=(730, 2), meta=np.ndarray>
Attributes:
    external_variables:     areacella
    history:                File was processed by fremetar (GFDL analog of CM...
    table_id:               day
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    comment:                <null ref>
    contact:                gfdl.climate.model.info@noaa.gov
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    creation_date:          2019-03-18T15:13:56Z
    data_specs_version:     01.00.27
    experiment:             update of RCP8.5 based on SSP5
    experiment_id:          ssp585
    forcing_index:          1
    frequency:              day
    further_info_url:       https://furtherinfo.es-doc.org/CMIP6.NOAA-GFDL.GF...
    grid:                   atmos data regridded from Cubed-sphere (c96) to 1...
    grid_label:             gr1
    initialization_index:   1
    institution:            National Oceanic and Atmospheric Administration, ...
    institution_id:         NOAA-GFDL
    license:                CMIP6 model data produced by NOAA-GFDL is license...
    mip_era:                CMIP6
    nominal_resolution:     100 km
    parent_activity_id:     CMIP
    parent_experiment_id:   historical
    parent_mip_era:         CMIP6
    parent_source_id:       GFDL-CM4
    parent_variant_label:   r1i1p1f1
    physics_index:          1
    product:                model-output
    realization_index:      1
    realm:                  atmos
    source:                 GFDL-CM4 (2018): \naerosol: interactive\natmos: G...
    source_id:              GFDL-CM4
    source_type:            AOGCM
    sub_experiment:         none
    sub_experiment_id:      none
    title:                  NOAA GFDL GFDL-CM4 model output prepared for CMIP...
    tracking_id:            hdl:21.14100/ad8c930c-124f-4b82-98b2-98cc4236ba25
    variable_id:            pr
    variant_info:           N/A
    references:             see further_info_url attribute
    variant_label:          r1i1p1f1
    branch_time_in_parent:  60225.0
    parent_time_units:      days since 1850-1-1

### How big do you make your chunks?

The rule of thumb for dask chunks is that you should "create arrays with a minimum chunksize of at least one million elements":  http://xarray.pydata.org/en/stable/dask.html#chunking-and-performance

netCDF4 chunks are often a lot smaller than dask array chunks. The minimum chunksize exists because if you have too many chunks, then queuing of operations when parallising will be slow, and if they are too big computation and memory can be wasted. The default chunks from dask gave us chunks of size: (7300, 180, 288) or nearly 400 million elements so we could try reducing those chunks if needed. The larger the array, the larger the cost of queueing and the larger chunks may be needed.

#### IMPORTANT: Whatever dask array chunks you use, make sure they align with the netCDF4 file chunks!!

So far our chunks have been in time, and the netCDF4 file is also chunked in time. If we tried to use dask chunks to optimise the time-series loading of data, it will not help! 

#### Exercise

Try it, load the data with chunks size `(31390,180,1)` (i.e. chunked in lon) and name that file `f_bad_chunk`. Try re-loading the time series of pr at `lat=0` and `lon=180` and time how long it takes.

In [18]:
f_bad_chunk = xr.open_mfdataset(path,chunks={'time':31390,'lat':180,'lon':1}, combine='by_coords')

In [19]:
%%time
f_bad_chunk.pr.sel(lat=0,lon=180,method='nearest').load()

CPU times: user 25.2 ms, sys: 9.91 ms, total: 35.1 ms
Wall time: 6.37 s


<xarray.DataArray 'pr' (time: 31390)>
array([2.1776618e-06, 9.6000194e-06, 5.3295435e-06, ..., 7.7904500e-05,
       1.6186357e-05, 1.5706388e-05], dtype=float32)
Coordinates:
    lon      float64 180.6
    lat      float64 0.5
  * time     (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
Attributes:
    long_name:      Precipitation
    units:          kg m-2 s-1
    cell_methods:   area: time: mean
    cell_measures:  area: areacella
    standard_name:  precipitation_flux
    interp_method:  conserve_order1
    original_name:  pr

What if we add some extra cores to the computation?

You can easily parallelise xarray code using the dask.distributed.Client and dask array calculations will be run in parallel.

In [20]:
from dask.distributed import Client
c = Client()
c

Client Scheduler: tcp://127.0.0.1:43603 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 32 Memory: 68.72 GB


#### Exercise

Try running your previous code for `f_bad_chunk` again loading the time series of pr at `lat=0` and `lon=180` and time how long it takes now that there are 8 workers in the dask cluster.

Do the same with the original chunking method of `f_ssp585` and see if there is a difference.

In [21]:
%%time
f_bad_chunk.pr.sel(lat=0,lon=180,method='nearest').load()

CPU times: user 486 ms, sys: 110 ms, total: 597 ms
Wall time: 7.11 s


<xarray.DataArray 'pr' (time: 31390)>
array([2.1776618e-06, 9.6000194e-06, 5.3295435e-06, ..., 7.7904500e-05,
       1.6186357e-05, 1.5706388e-05], dtype=float32)
Coordinates:
    lon      float64 180.6
    lat      float64 0.5
  * time     (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
Attributes:
    long_name:      Precipitation
    units:          kg m-2 s-1
    cell_methods:   area: time: mean
    cell_measures:  area: areacella
    standard_name:  precipitation_flux
    interp_method:  conserve_order1
    original_name:  pr

In [22]:
%%time
f_ssp585.pr.sel(lat=0,lon=180,method='nearest').load()

CPU times: user 541 ms, sys: 103 ms, total: 644 ms
Wall time: 4.37 s


<xarray.DataArray 'pr' (time: 31390)>
array([2.1776618e-06, 9.6000194e-06, 5.3295435e-06, ..., 7.7904500e-05,
       1.6186357e-05, 1.5706388e-05], dtype=float32)
Coordinates:
    lon      float64 180.6
    lat      float64 0.5
  * time     (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
Attributes:
    long_name:      Precipitation
    units:          kg m-2 s-1
    cell_methods:   area: time: mean
    cell_measures:  area: areacella
    standard_name:  precipitation_flux
    interp_method:  conserve_order1
    original_name:  pr

### Poor chunking with dask can make your performance worse!

Both the size of the chunks and the alignment of the chunks with the filesystem chunks are imporant to keep in mind when creating dask chunks. 

### Summary

This example shows how to make data chunking with dask. 

For further information regarding Dask please see: https://docs.dask.org/en/latest/